In [1]:
import os 
from dotenv import load_dotenv
load_dotenv('C:/Users/midon/Documents/anomaly-detection-autoencoder-based-basic/.env.local')
PROJECT_PATH = os.getenv('PROJECT_PATH')
os.chdir(PROJECT_PATH)
from preprocessing.preprocessing import * 
path = 'synthetic_data/synthetic_data.csv'
synth = SyntheticData(path)
X = synth.get_X_train(array_format=True)
# synth.bool_cols
synth_ft = feature_engineer(synth)
X_ft = synth_ft.get_X_train(array_format = True)
y = synth.y


c:\Users\midon\Documents\anomaly-detection-autoencoder-based-basic\.venv\Lib\site-packages\pandas\core\internals\blocks.py:395: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


Applying Yeo-Johnson transformations...


In [20]:
print(f'Number of features before feature engineering: {X.shape[1]}')
print(f'Number of features after feature engineering: {X_ft.shape[1]}')

Number of features before feature engineering: 50
Number of features after feature engineering: 45


After feature engineering is applied, the isolation forest (with the same hyperparams) could detect default loan better (as outliers) --> Feature engineering is good.

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)
X_train_ft, X_test_ft, y_train_ft, y_test_ft = train_test_split(
        X_ft, y, test_size=0.2, random_state=42)

In [3]:
def transform_iso_prediction(y:np.array):
    pred = y.copy()
    pred[pred==1] = 0 
    pred[pred==-1] = 1
    return pred

In [25]:
from sklearn.ensemble import IsolationForest
from eval.evaluate_recon import evaluate_metrics,confusion_matrix_metrics
iso = IsolationForest(n_estimators=200,contamination=0.3, max_features=20) #change max_features but still the same 
iso.fit(X_train)
pred_train = iso.predict(X_train)
pred_train = transform_iso_prediction(pred_train)
pred_test = iso.predict(X_test)
pred_test = transform_iso_prediction(pred_test)
print('Results on train and test set before feature engineering')
evaluate_metrics(y_train, pred_train)
confusion_matrix_metrics(y_train, pred_train)
evaluate_metrics(y_test, pred_test)
confusion_matrix_metrics(y_test, pred_test)

Results on train and test set before feature engineering
F1 Score: 0.2378, Precision: 0.2049, Recall: 0.2833, Accuracy: 0.606
TN: 101572, FP: 44494, FN: 28994, TP: 11463
F1 Score: 0.2409, Precision: 0.2076, Recall: 0.287, Accuracy: 0.6062
TN: 25352, FP: 11125, FN: 7240, TP: 2914


(25352, 11125, 7240, 2914)

In [26]:
iso_ft = IsolationForest(n_estimators=200,contamination=0.3, max_features=20)
iso_ft.fit(X_train_ft)
pred_train_ft = iso_ft.predict(X_train_ft)
pred_train_ft = transform_iso_prediction(pred_train_ft)
pred_test_ft = iso_ft.predict(X_test_ft)
pred_test_ft = transform_iso_prediction(pred_test_ft)
print('Results on train and test set after feature engineering')
evaluate_metrics(y_train_ft, pred_train_ft)
confusion_matrix_metrics(y_train_ft, pred_train_ft)
evaluate_metrics(y_test_ft, pred_test_ft)
confusion_matrix_metrics(y_test_ft, pred_test_ft)

Results on train and test set after feature engineering
F1 Score: 0.2507, Precision: 0.216, Recall: 0.2987, Accuracy: 0.6127
TN: 102193, FP: 43873, FN: 28373, TP: 12084
F1 Score: 0.2542, Precision: 0.2188, Recall: 0.3033, Accuracy: 0.6125
TN: 25480, FP: 10997, FN: 7074, TP: 3080


(25480, 10997, 7074, 3080)